# Re-Chunking Larger Datasets 

This notebook extends ideas covered in the [basic workflow](./ReChunkingData.ipynb).  This 
notebook will perfrom the same operations, but will work on the **much** larger dataset, and 
involve some parallelization using the dask scheduler. 

:::{Warning}

You should run this **only** on a cloud compute node -- on ESIP Nebari, for example. We 
will be reading and writing **enormous** amounts of data to S3 buckets. To do that over a 
typical network connection will saturate your bandwidth and take days to complete.

:::

## System Setup 

In [1]:
# Activate logging
import logging
logging.basicConfig(level=logging.INFO, force=True)


## Plumb Data Source
We're going to look at a particular dataset from the National Water Model Reanalysis Version 2.1. 
The dataset is part of the AWS Open Data Program, and is included in the HyTEST data catalog.


In [2]:
import intake
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'
cat = intake.open_catalog(url)
cat['nwm21-streamflow-cloud']

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


nwm21-streamflow-cloud:
  args:
    consolidated: true
    storage_options:
      anon: true
    urlpath: s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr
  description: National Water Model 2.1 CHRTOUT on AWS
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog


## Load the zarr data
We'll take advantage of the `intake` mechanism and load the data 
directly.  We'll need to set up our AWS credentials first, since
this data is stored on an S3 bucket. 

In [3]:
import os
os.environ['AWS_PROFILE'] = 'osn-renci'
%run ../environment_set_up/Help_AWS_Credentials.ipynb

ds = cat['nwm21-streamflow-cloud'].to_dask()
import xarray as xr


## Spin up Dask Cluster
Our rechunking operation will be able to work in parallel. To do that, we will
spin up a `dask` cluster on the cloud hardware to schedule the various workers.
Note that this cluster must be configured with a specific user **profile** with 
permissions to write to our eventual output location. 

In [22]:
# %run ../environment_set_up/Start_Dask_Cluster_Nebari.ipynb
import os
import logging

try:
    from dask_gateway import Gateway
except ImportError:
    logging.error("Unable to import Dask Gateway.  Are you running in a cloud compute environment?\n")
    raise
os.environ['DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION'] = "1.0"

gateway = Gateway()
_options = gateway.cluster_options()
_options.conda_environment='users/users-pangeo'  ##<< this is the conda environment we use on nebari.
_options.profile = 'Medium Worker'
_env_to_add={}
aws_env_vars=['AWS_ACCESS_KEY_ID',
              'AWS_SECRET_ACCESS_KEY',
              'AWS_SESSION_TOKEN',
              'AWS_DEFAULT_REGION',
              'AWS_S3_ENDPOINT', 
              #'AWS_PROFILE',
              'DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION']
for _e in aws_env_vars:
    if _e in os.environ:
        _env_to_add[_e] = os.environ[_e]
_options.environment_vars = _env_to_add    
cluster = gateway.new_cluster(_options)          ##<< create cluster via the dask gateway
cluster.adapt(minimum=2, maximum=30)             ##<< Sets scaling parameters. 

client = cluster.get_client()

print("The 'cluster' object can be used to adjust cluster behavior.  i.e. 'cluster.adapt(minimum=10)'")
print("The 'client' object can be used to directly interact with the cluster.  i.e. 'client.submit(func)' ")
print(f"The link to view the client dashboard is:\n>  {client.dashboard_link}")

The 'cluster' object can be used to adjust cluster behavior.  i.e. 'cluster.adapt(minimum=10)'
The 'client' object can be used to directly interact with the cluster.  i.e. 'client.submit(func)' 
The link to view the client dashboard is:
>  https://nebari.esipfed.org/gateway/clusters/dev.f96ef9e477fb4d61bc1ddb885a1b6722/status


## Read Sample Data

In [23]:
import dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    smplData = ds.where(ds.gage_id != ''.rjust(15).encode(), drop=True) # subset to only those features with a valid gage_id
    smplData.drop('crs') # Not needed/wanted for this analysis
smplData

<xarray.Dataset>
Dimensions:     (feature_id: 7994, time: 367439)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 12932 ... 1170023539 1180000535
    gage_id     (feature_id) |S15 dask.array<chunksize=(7994,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    crs         (feature_id) object dask.array<chunksize=(7994,), meta=np.ndarray>
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 106), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 106), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

## Re-Chunk Plan
We will configure a new chunking plan which will favor time-series analysis. 
Using the dimensions of the data: 
* 367439 time steps
* 7994 feature IDs

We can write the new plan as: 

In [24]:
# The new chunking plan:
chunk_plan = {
    'streamflow': {'time': 367439, 'feature_id': 1}, # all time records in one chunk for each feature_id
    'velocity': {'time': 367439, 'feature_id': 1},
    'elevation': (7994,),
    'gage_id': (7994,),
    'latitude': (7994,),
    'longitude': (7994,),    
    'order': (7994,),    
    'time': (367439,), # all time coordinates in one chunk
    'feature_id': (7994,) # all feature_id coordinates in one chunk
}


In [25]:
del os.environ['DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION']

In [26]:
# Manually reset the chunking metadata in prep for re-chunking
#smplData = smplData.chunk(chunks={'feature_id':1, 'time': 367439})
for x in smplData.variables:
    smplData[x].encoding['chunks'] = None

## Set up output location

With this plan, we can ask `rechunker` to re-write the data using the prescribed chunking pattern.

Unlike with the smaller dataset, we need to write this very large dataset to an object store in the datacenter: an S3 'bucket'.  So we need to set that up so that `rechunker` will have a suitable place to write data. This new data will be a complete copy of the original, just re-organized a bit. 

In [27]:
from getpass import getuser
import fsspec
uname=getuser()

fsw = fsspec.filesystem(
    's3', 
    anon=False, 
    default_fill_cache=False, 
    skip_instance_cache=True, 
    client_kwargs={'endpoint_url': os.environ['AWS_S3_ENDPOINT']}
)
workspace = 's3://rsignellbucket2/'
testDir = workspace + "testing/"
myDir = testDir + f'{uname}_ReChunkTutorial/'
fsw.ls(testDir)

INFO:aiobotocore.credentials:Found credentials in environment variables.


['rsignellbucket2/testing/02_kerchunk.ipynb',
 'rsignellbucket2/testing/ATL08_20181014084920_02400109_003_01.h5',
 'rsignellbucket2/testing/cluster_conf.py',
 'rsignellbucket2/testing/cog',
 'rsignellbucket2/testing/combine_files_tpBiasCorr.csh',
 'rsignellbucket2/testing/foo.json',
 'rsignellbucket2/testing/gzt5142',
 'rsignellbucket2/testing/ortho_2021_10_7_sageLot_noalpha_clip_cog.tif',
 'rsignellbucket2/testing/time']

In [28]:
for f in ['rechunked.zarr', 'staging.zarr']:
    if fsw.exists(myDir + f):
        fsw.rm(myDir + f, recursive=True)
staging = fsw.get_mapper(myDir + 'staging.zarr', create=True)
outfile = fsw.get_mapper(myDir + 'rechunked.zarr', create=True)

## Ready to rechunk

In [37]:
import rechunker
import fsspec
## Recall that merely invoking rechunker does not do any work... just sorts out 
## the rechunking plan and writes metadata.
result = rechunker.rechunk(
    smplData,
    chunk_plan,
    "2GB",
    outfile, 
    temp_store=staging 
)

ReadOnlyError: object is read-only

In [ ]:
from dask.distributed import progress, performance_report

with performance_report(filename="dask-report.html"):
    r = result.execute(retries=10)  

In [ ]:
import zarr
_ = zarr.consolidate_metadata(outfile)

## Results
Let's read in the resulting re-chunked dataset to see how it looks:

In [34]:
reChunkedData = xr.open_zarr(outfile)
reChunkedData

FileNotFoundError: No such file or directory: '<fsspec.mapping.FSMap object at 0x7f151aa90430>'

### Comparison


In [ ]:
## Before:
sampleData['streamflow'].sel(feature_id=1343034)
# Note: three chunks needed to service a single feature_id


In [ ]:
## After:
reChunkedData['streamflow'].sel(feature_id=1343034) 
# All data for the specified feature_id is in a single chunk


In [21]:
client.close()
cluster.close()

In [18]:

def myAWS_Credentials():
    """Test function to return AWS credential information."""
    return {
    "AWS_PROFILE": os.environ.get("AWS_PROFILE", "<not set>"),
    "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID', '<not set>'),
    "AWS_S3_ENDPOINT": os.environ.get('AWS_S3_ENDPOINT', '<not set>')    
}


In [19]:
myAWS_Credentials()

{'AWS_PROFILE': 'osn-renci',
 'AWS_ACCESS_KEY_ID': '8A852VG4EG6NHHM4WUDX',
 'AWS_S3_ENDPOINT': 'https://renc.osn.xsede.org'}

In [20]:
client.submit(myAWS_Credentials).result()

{'AWS_PROFILE': 'osn-renci',
 'AWS_ACCESS_KEY_ID': '8A852VG4EG6NHHM4WUDX',
 'AWS_S3_ENDPOINT': 'https://renc.osn.xsede.org'}

In [31]:
fsw.touch(myDir + "dummyfile.txt")

{'ResponseMetadata': {'RequestId': 'tx00000afcda2c18bbcd1f3-00640762cd-bbfdf-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx00000afcda2c18bbcd1f3-00640762cd-bbfdf-default',
   'date': 'Tue, 07 Mar 2023 16:14:06 GMT'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [32]:
client.submit(fsw.touch, myDir+"dd.txt").result()

{'ResponseMetadata': {'RequestId': 'tx000006f354b826b0d5589-00640762f2-bbfdf-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '0',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx000006f354b826b0d5589-00640762f2-bbfdf-default',
   'date': 'Tue, 07 Mar 2023 16:14:42 GMT'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [33]:
fsw.ls(myDir)

['rsignellbucket2/testing/gzt5142_ReChunkTutorial/dd.txt',
 'rsignellbucket2/testing/gzt5142_ReChunkTutorial/dummyfile.txt']

In [2]:
import fsspec
fsspec.__version__

'2023.1.0+16.gd648c3b'

In [1]:
import fsspec
fsspec.__version__

'2023.3.0'